## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..//Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Longonjo,AO,-12.9067,15.2533,58.51,59,1,2.84,clear sky
1,1,Hermanus,ZA,-34.4187,19.2345,59.81,54,56,8.50,broken clouds
2,2,Sobolevo,RU,54.4299,31.8996,46.67,96,32,8.03,scattered clouds
3,3,Busselton,AU,-33.6500,115.3333,45.09,63,81,18.79,broken clouds
4,4,Dharchula,IN,29.8500,80.5333,66.58,70,68,5.82,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Hilo,US,19.7297,-155.0900,79.79,90,7,7.00,clear sky
11,11,Alofi,NU,-19.0595,-169.9187,80.19,76,68,8.84,broken clouds
14,14,Atuona,PF,-9.8000,-139.0333,79.57,80,25,19.66,light rain
16,16,Georgetown,MY,5.4112,100.3354,82.89,86,20,1.99,few clouds
20,20,Luganville,VU,-15.5333,167.1667,77.72,75,100,8.03,overcast clouds
25,25,Sabang,ID,5.8933,95.3214,84.15,75,100,16.53,overcast clouds
27,27,Puerto Ayora,EC,-0.7393,-90.3518,77.56,83,68,7.00,broken clouds
29,29,Bathsheba,BB,13.2167,-59.5167,80.80,74,55,16.58,broken clouds
31,31,Kutum,SD,14.2000,24.6667,76.80,16,4,13.15,clear sky
38,38,Butaritari,KI,3.0707,172.7902,82.89,78,71,17.87,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                186
City                   186
Country                184
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Current Description    186
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')
clean_df.count()

City_ID                184
City                   184
Country                184
Lat                    184
Lng                    184
Max Temp               184
Humidity               184
Cloudiness             184
Wind Speed             184
Current Description    184
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Hilo,US,79.79,clear sky,19.7297,-155.0900,
11,Alofi,NU,80.19,broken clouds,-19.0595,-169.9187,
14,Atuona,PF,79.57,light rain,-9.8000,-139.0333,
16,Georgetown,MY,82.89,few clouds,5.4112,100.3354,
20,Luganville,VU,77.72,overcast clouds,-15.5333,167.1667,
25,Sabang,ID,84.15,overcast clouds,5.8933,95.3214,
27,Puerto Ayora,EC,77.56,broken clouds,-0.7393,-90.3518,
29,Bathsheba,BB,80.80,broken clouds,13.2167,-59.5167,
31,Kutum,SD,76.80,clear sky,14.2000,24.6667,
38,Butaritari,KI,82.89,light rain,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": ""
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Hilo,US,79.79,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
11,Alofi,NU,80.19,broken clouds,-19.0595,-169.9187,Taloa Heights
14,Atuona,PF,79.57,light rain,-9.8000,-139.0333,Villa Enata
16,Georgetown,MY,82.89,few clouds,5.4112,100.3354,Cititel Penang
20,Luganville,VU,77.72,overcast clouds,-15.5333,167.1667,The Espiritu Hotel
...,...,...,...,...,...,...,...
668,Aitape,PG,79.32,light rain,-3.1374,142.3475,
671,Ishigaki,JP,82.65,broken clouds,24.3448,124.1572,Hotel Peace Land Ishigakijima
676,Pangai,TO,77.97,overcast clouds,-19.8000,-174.3500,Ha'apai Beach Resort
681,Bassila,BJ,77.59,light rain,9.0081,1.6654,COMPLEXE MAKOYA ET FILS


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))